In [34]:
import pandas as pd
import numpy as np
import chart_studio as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

import plotly.graph_objects as go
import pprint as pp


In [35]:
from quantfreedom._typing import pdFrame

In [36]:
def generate_candles(
    number_of_candles: int = 100,
    seed: int = None,
) -> pdFrame:
    """
    Generate a dataframe filled with random candles

    Explainer Video
    ---------------
        Coming_Soon
    
    Parameters
    ----------
    number_of_candles: int = 100
        number of candles you want to create
    seed: int = None
        random seed number

    Returns
    -------
    pdFrame
        Dataframe of open high low close
    """
    np.random.seed(seed)

    periods = number_of_candles * 48

    prices = np.around(5000 + np.random.normal(scale=1.5, size=periods).cumsum(), 2)

    data = pd.DataFrame(
        prices,
        index=pd.Index(
            pd.date_range("01/01/2000", periods=periods, freq="30min"),
            name="open_time",
        ),
        columns=["price"],
    )
    data = data.price.resample("D").ohlc()

    data.columns = pd.MultiIndex.from_tuples(
        tuples=[
            ("QuantFreedom", "open"),
            ("QuantFreedom", "high"),
            ("QuantFreedom", "low"),
            ("QuantFreedom", "close"),
        ],
        name=["symbol", "candle_info"],
    )
    fig = go.Figure(
        data=go.Candlestick(
            x=data.index,
            open=data.iloc[:, 0],
            high=data.iloc[:, 1],
            low=data.iloc[:, 2],
            close=data.iloc[:, 3],
        )
    )
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()

    return data

generated_data = generate_candles(number_of_candles=100, seed=42)
generated_data

symbol      QuantFreedom                           
candle_info         open     high      low    close
open_time                                          
2000-01-01      5,000.75 5,006.72 4,983.63 4,985.22
2000-01-02      4,985.73 4,991.00 4,981.63 4,983.93
2000-01-03      4,984.38 4,989.74 4,979.71 4,982.52
2000-01-04      4,982.91 4,994.96 4,980.25 4,991.20
2000-01-05      4,991.52 5,003.96 4,987.77 4,999.14
...                  ...      ...      ...      ...
2000-04-05      5,045.46 5,056.78 5,044.76 5,054.98
2000-04-06      5,056.01 5,056.01 5,039.46 5,046.95
2000-04-07      5,046.47 5,048.44 5,041.20 5,043.57
2000-04-08      5,044.43 5,057.13 5,044.24 5,053.64
2000-04-09      5,053.29 5,053.29 5,033.93 5,035.04

[100 rows x 4 columns]

In [37]:
generated_data['QuantFreedom']['close']
close_prices = generated_data['QuantFreedom']['close']
high_prices = generated_data['QuantFreedom']['high']
low_prices = generated_data['QuantFreedom']['low']


In [38]:
def calculate_fractal(data):
    high = high_prices
    # low = low_prices
    
    fractal_up = high.rolling(window=8, center=True).apply(lambda x: x[1] > x[0] and x[1] > x[2])
    # fractal_down = low.rolling(window=3, center=True).apply(lambda x: x[1] < x[0] and x[1] < x[2])
    
    return fractal_up
""", fractal_down"""


def calculate_fractal_2(data):
    high = high_prices
    # low = data['Low']
    
    fractal_up = (high.shift(1) < high) & (high.shift(-1) < high) & (high.shift(2) < high) & (high.shift(-2) < high)
    # fractal_down = (low.shift(1) > low) & (low.shift(-1) > low) & (low.shift(2) > low) & (low.shift(-2) > low)
    
    return fractal_up
# , fractal_down

fractal_signal = calculate_fractal(generated_data)

fractal_signal

open_time
2000-01-01    NaN
2000-01-02    NaN
2000-01-03    NaN
2000-01-04    NaN
2000-01-05   0.00
             ... 
2000-04-05   0.00
2000-04-06   0.00
2000-04-07    NaN
2000-04-08    NaN
2000-04-09    NaN
Freq: D, Name: high, Length: 100, dtype: float64

In [39]:
fractal_signal2 = calculate_fractal_2(generated_data)

fractal_signal2

open_time
2000-01-01    False
2000-01-02    False
2000-01-03    False
2000-01-04    False
2000-01-05    False
              ...  
2000-04-05     True
2000-04-06    False
2000-04-07    False
2000-04-08    False
2000-04-09    False
Freq: D, Name: high, Length: 100, dtype: bool

In [40]:
print(fractal_signal.to_string())


open_time
2000-01-01    NaN
2000-01-02    NaN
2000-01-03    NaN
2000-01-04    NaN
2000-01-05   0.00
2000-01-06   0.00
2000-01-07   0.00
2000-01-08   1.00
2000-01-09   0.00
2000-01-10   0.00
2000-01-11   0.00
2000-01-12   1.00
2000-01-13   0.00
2000-01-14   0.00
2000-01-15   0.00
2000-01-16   0.00
2000-01-17   1.00
2000-01-18   0.00
2000-01-19   0.00
2000-01-20   0.00
2000-01-21   0.00
2000-01-22   0.00
2000-01-23   0.00
2000-01-24   0.00
2000-01-25   0.00
2000-01-26   0.00
2000-01-27   0.00
2000-01-28   0.00
2000-01-29   1.00
2000-01-30   0.00
2000-01-31   1.00
2000-02-01   0.00
2000-02-02   0.00
2000-02-03   0.00
2000-02-04   1.00
2000-02-05   0.00
2000-02-06   0.00
2000-02-07   0.00
2000-02-08   0.00
2000-02-09   1.00
2000-02-10   0.00
2000-02-11   0.00
2000-02-12   0.00
2000-02-13   0.00
2000-02-14   1.00
2000-02-15   0.00
2000-02-16   0.00
2000-02-17   0.00
2000-02-18   0.00
2000-02-19   1.00
2000-02-20   0.00
2000-02-21   1.00
2000-02-22   0.00
2000-02-23   0.00
2000-02-24   0.00


In [41]:
# # Convert fractal_signal to boolean
# fractal_signal = fractal_signal.astype(bool)

In [42]:
# Assuming `fractal_signal` is a Pandas Series
fractal_signal = fractal_signal.fillna(0.0)  # replace NaN with 0.0
fractal_signal = fractal_signal.replace({True: 1.0, False: 0.0})  # convert True to 1.0 and False to 0.0
fractal_signal = fractal_signal.astype(bool)  # convert to boolean

In [43]:
fractal_signal

open_time
2000-01-01    False
2000-01-02    False
2000-01-03    False
2000-01-04    False
2000-01-05    False
              ...  
2000-04-05    False
2000-04-06    False
2000-04-07    False
2000-04-08    False
2000-04-09    False
Freq: D, Name: high, Length: 100, dtype: bool

In [44]:
type(fractal_signal)

pandas.core.series.Series

In [45]:
print(fractal_signal.to_string())


open_time
2000-01-01    False
2000-01-02    False
2000-01-03    False
2000-01-04    False
2000-01-05    False
2000-01-06    False
2000-01-07    False
2000-01-08     True
2000-01-09    False
2000-01-10    False
2000-01-11    False
2000-01-12     True
2000-01-13    False
2000-01-14    False
2000-01-15    False
2000-01-16    False
2000-01-17     True
2000-01-18    False
2000-01-19    False
2000-01-20    False
2000-01-21    False
2000-01-22    False
2000-01-23    False
2000-01-24    False
2000-01-25    False
2000-01-26    False
2000-01-27    False
2000-01-28    False
2000-01-29     True
2000-01-30    False
2000-01-31     True
2000-02-01    False
2000-02-02    False
2000-02-03    False
2000-02-04     True
2000-02-05    False
2000-02-06    False
2000-02-07    False
2000-02-08    False
2000-02-09     True
2000-02-10    False
2000-02-11    False
2000-02-12    False
2000-02-13    False
2000-02-14     True
2000-02-15    False
2000-02-16    False
2000-02-17    False
2000-02-18    False
2000-02-19

In [46]:
generated_data
close_prices = generated_data['QuantFreedom']['close']
high_prices = generated_data['QuantFreedom']['high']
low_prices = generated_data['QuantFreedom']['low']
open_prices = generated_data['QuantFreedom']['open']
index_generated_data = generated_data.index

In [47]:
# print index column from generated_data
generated_data.index



DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14', '2000-01-15', '2000-01-16',
               '2000-01-17', '2000-01-18', '2000-01-19', '2000-01-20',
               '2000-01-21', '2000-01-22', '2000-01-23', '2000-01-24',
               '2000-01-25', '2000-01-26', '2000-01-27', '2000-01-28',
               '2000-01-29', '2000-01-30', '2000-01-31', '2000-02-01',
               '2000-02-02', '2000-02-03', '2000-02-04', '2000-02-05',
               '2000-02-06', '2000-02-07', '2000-02-08', '2000-02-09',
               '2000-02-10', '2000-02-11', '2000-02-12', '2000-02-13',
               '2000-02-14', '2000-02-15', '2000-02-16', '2000-02-17',
               '2000-02-18', '2000-02-19', '2000-02-20', '2000-02-21',
               '2000-02-22', '2000-02-23', '2000-02-24', '2000-02-25',
      

In [48]:
# create the price chart
# fig = go.Figure(data=[go.Scatter(y=prices)])
fig = go.Figure(data=[go.Candlestick(x=index_generated_data, open=open_prices, high=high_prices, low=low_prices, close=close_prices)])


# add the Boolean signal as a scatter trace
signal_points = [i for i, sig in enumerate(fractal_signal) if sig]
fig.add_trace(go.Scatter(x=[index_generated_data[i] for i in signal_points], y=[low_prices[i] for i in signal_points], mode='markers', marker=dict(color='red', size=10)))

# show the chart
fig.show()



# add the Boolean signal as a scatter trace
# signal_points = [i for i, sig in enumerate(fractal_signal) if sig]
# fig.add_trace(go.Scatter(x=signal_points, y=[prices[i] for i in signal_points], mode='markers', marker=dict(color='red', size=10)))

# # show the chart
# fig.show()

In [51]:
# create the price chart
# fig = go.Figure(data=[go.Scatter(y=prices)])
fig = go.Figure(data=[go.Candlestick(x=index_generated_data, open=open_prices, high=high_prices, low=low_prices, close=close_prices)])


# add the Boolean signal as a scatter trace
signal_points = [i for i, sig in enumerate(fractal_signal2) if sig]
fig.add_trace(go.Scatter(x=[index_generated_data[i] for i in signal_points], y=[high_prices[i] for i in signal_points], mode='markers', marker=dict(color='yellow', size=10)))

# show the chart
fig.show()



# add the Boolean signal as a scatter trace
# signal_points = [i for i, sig in enumerate(fractal_signal) if sig]
# fig.add_trace(go.Scatter(x=signal_points, y=[prices[i] for i in signal_points], mode='markers', marker=dict(color='red', size=10)))

# # show the chart
# fig.show()